In [1]:
import requests
from datetime import datetime
import time
import json
import glob

import pandas as pd
import numpy as np
import geopandas as gpd
from pytz import timezone
%matplotlib inline
import seaborn as sns
from tqdm import tqdm_notebook as tqdm

In [2]:
api_key = 'ac31e02046794bd3f959fbc98fc25231'

In [3]:
gdf = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_2.json')
center = gpd.GeoDataFrame(gdf[['GID_2', 'NAME_2']])
center['geometry'] = gdf.centroid
center = center.to_crs(gdf.crs)
center['lat'] = center.geometry.y
center['lon'] = center.geometry.x
center
# gdf

C:\Users\sonle\AppData\Local\Temp\ipykernel_38400\2083312229.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center['geometry'] = gdf.centroid
C:\Users\sonle\AppData\Local\Temp\ipykernel_38400\2083312229.py:3: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  center['geometry'] = gdf.centroid


,GID_2,NAME_2,geometry,lat,lon
0,SRB.1.1_1,Bor,POINT (22.05425 44.07092),44.070917,22.054249
1,SRB.1.2_1,Kladovo,POINT (22.47141 44.56865),44.568655,22.471406
2,SRB.1.3_1,Majdanpek,POINT (22.058 44.38801),44.388008,22.057996
3,SRB.1.4_1,Negotin,POINT (22.42711 44.24799),44.247986,22.427111
4,SRB.2.1_1,Golubac,POINT (21.74048 44.60889),44.608886,21.740484
...,...,...,...,...,...
156,SRB.25.6_1,Požega,POINT (20.0178 43.88806),43.888056,20.017801
157,SRB.25.7_1,Priboj,POINT (19.40003 43.50832),43.508319,19.400028
158,SRB.25.8_1,Prijepolje,POINT (19.62589 43.32048),43.320476,19.625891
159,SRB.25.9_1,Sjenica,POINT (20.00164 43.20967),43.209667,20.001638


In [4]:
gid_2 = center['GID_2'].to_list()
lat = center['lat'].to_list()
lon = center['lon'].to_list()

loc_dict = {}
for i in range(len(gid_2)):
    loc_dict[gid_2[i]] = [lat[i], lon[i]]

In [5]:
loc_dict

{'SRB.1.1_1': [44.070916840003875, 22.05424872664738],
 'SRB.1.2_1': [44.5686546148656, 22.471405549626134],
 'SRB.1.3_1': [44.38800848123445, 22.057996177042046],
 'SRB.1.4_1': [44.247985541215726, 22.42711139779415],
 'SRB.2.1_1': [44.60888632367149, 21.74048439329081],
 'SRB.2.2_1': [44.45935440783599, 21.714726353653496],
 'SRB.2.3_1': [44.52122465631768, 21.335332314037128],
 'SRB.2.4_1': [44.337986357280606, 21.439651473279095],
 'SRB.2.5_1': [44.63989642646239, 21.190605805795062],
 'SRB.2.6_1': [44.687345803589444, 21.43636807884723],
 'SRB.2.7_1': [44.36787554880919, 21.214356338737765],
 'SRB.2.8_1': [44.19508760637038, 21.735714704002095],
 'SRB.3.1_1': [44.549994492875925, 20.371487647191852],
 'SRB.3.2_1': [44.68998770604238, 20.361058549735947],
 'SRB.3.3_1': [44.634950376453595, 20.652155765778986],
 'SRB.3.4_1': [44.37768730613215, 20.3008821128093],
 'SRB.3.5_1': [44.42604587242706, 20.681950629318763],
 'SRB.3.6_1': [44.79295725163482, 20.36386308456866],
 'SRB.3.7_1'

In [7]:
def create_json(loc:str, start: int, end: int) -> dict:
    lat = loc_dict[loc][0]
    lon = loc_dict[loc][1]
    hist_url = f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}'
    r = requests.get(hist_url)
    json_out = r.json()
    json_out['name'] = loc

    return json_out

year = 2022
start_date = int(time.mktime(datetime(year, 1, 1).timetuple()))
end_date = int(time.mktime(datetime(year, 12, 31).timetuple()))

for location_name in tqdm(list(loc_dict.keys())):
    print(location_name)
    json_res = create_json(location_name, start_date, end_date)
    with open(f"D:\Work\WB\LDT\countries\SRB\\raw_data\\air_pollution\municipalities\{location_name}_{year}.json", "w") as outfile: 
            json.dump(json_res, outfile)



C:\Users\sonle\AppData\Local\Temp\ipykernel_38400\1651517222.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for location_name in tqdm(list(loc_dict.keys())):


  0%|          | 0/161 [00:00<?, ?it/s]

SRB.1.1_1
SRB.1.2_1
SRB.1.3_1
SRB.1.4_1
SRB.2.1_1
SRB.2.2_1
SRB.2.3_1
SRB.2.4_1
SRB.2.5_1
SRB.2.6_1
SRB.2.7_1
SRB.2.8_1
SRB.3.1_1
SRB.3.2_1
SRB.3.3_1
SRB.3.4_1
SRB.3.5_1
SRB.3.6_1
SRB.3.7_1
SRB.3.8_1
SRB.3.9_1
SRB.3.10_1
SRB.3.11_1
SRB.3.12_1
SRB.3.13_1
SRB.3.14_1
SRB.3.15_1
SRB.3.16_1
SRB.3.17_1
SRB.4.1_1
SRB.4.2_1
SRB.4.3_1
SRB.4.4_1
SRB.4.5_1
SRB.4.6_1
SRB.5.1_1
SRB.5.2_1
SRB.5.3_1
SRB.5.4_1
SRB.5.5_1
SRB.5.6_1
SRB.5.7_1
SRB.5.8_1
SRB.5.9_1
SRB.5.10_1
SRB.5.11_1
SRB.5.12_1
SRB.6.1_1
SRB.6.2_1
SRB.6.3_1
SRB.6.4_1
SRB.6.5_1
SRB.6.6_1
SRB.6.7_1
SRB.6.8_1
SRB.7.1_1
SRB.7.2_1
SRB.7.3_1
SRB.7.4_1
SRB.7.5_1
SRB.7.6_1
SRB.8.1_1
SRB.8.2_1
SRB.8.3_1
SRB.8.4_1
SRB.8.5_1
SRB.8.6_1
SRB.8.7_1
SRB.8.8_1
SRB.9.1_1
SRB.9.2_1
SRB.9.3_1
SRB.9.4_1
SRB.10.1_1
SRB.10.2_1
SRB.10.3_1
SRB.10.4_1
SRB.10.5_1
SRB.10.6_1
SRB.10.7_1
SRB.11.1_1
SRB.11.2_1
SRB.11.3_1
SRB.11.4_1
SRB.11.5_1
SRB.11.6_1
SRB.11.7_1
SRB.12.1_1
SRB.12.2_1
SRB.12.3_1
SRB.12.4_1
SRB.13.1_1
SRB.13.2_1
SRB.13.3_1
SRB.14.1_1
SRB.14.2_1
SRB.14

In [111]:
# gdf = gpd.read_file('countries/SRB/gadm41_SRB_2.json')
# center = gpd.GeoDataFrame(gdf[['GID_2', 'NAME_2']])
# center['geometry'] = gdf.centroid
# center = center.to_crs(gdf.crs)
# center['lat'] = center.geometry.y
# center['lon'] = center.geometry.x

# gid_2 = center['GID_2'].to_list()
# lat = center['lat'].to_list()
# lon = center['lon'].to_list()

# loc_dict = {}
# for i in range(len(gid_2)):
#     loc_dict[gid_2[i]] = [lat[i], lon[i]]

# loc_dict

In [112]:
# def create_json(loc:str, start: int, end: int) -> dict:
#     lat = loc_dict[loc][0]
#     lon = loc_dict[loc][1]
#     hist_url = f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}'
#     r = requests.get(hist_url)
#     json_out = r.json()
#     json_out['name'] = loc

#     return json_out

# year = 2023
# start_date = int(time.mktime(datetime(year, 1, 1).timetuple()))
# end_date = int(time.mktime(datetime(year, 12, 31).timetuple()))

# for location_name in list(loc_dict.keys()):
#     print(location_name)
#     json_res = create_json(location_name, start_date, end_date)
#     with open(f"countries/SRB/air_pollution/municipalities/{location_name}_{year}.json", "w") as outfile: 
#             json.dump(json_res, outfile)




In [11]:
paths = glob.glob('D:\Work\WB\LDT\countries\SRB\\raw_data\\air_pollution\municipalities/*.json')
paths.sort()
paths

['D:\\Work\\WB\\LDT\\countries\\SRB\\raw_data\\air_pollution\\municipalities\\SRB.1.1_1_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\raw_data\\air_pollution\\municipalities\\SRB.1.2_1_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\raw_data\\air_pollution\\municipalities\\SRB.1.3_1_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\raw_data\\air_pollution\\municipalities\\SRB.1.4_1_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\raw_data\\air_pollution\\municipalities\\SRB.10.1_1_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\raw_data\\air_pollution\\municipalities\\SRB.10.2_1_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\raw_data\\air_pollution\\municipalities\\SRB.10.3_1_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\raw_data\\air_pollution\\municipalities\\SRB.10.4_1_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\raw_data\\air_pollution\\municipalities\\SRB.10.5_1_2023.json',
 'D:\\Work\\WB\\LDT\\countries\\SRB\\raw_data\\air_pollution\\municipalities\\SRB.10.6_1_2023.j

In [9]:
dfs = []

for path in tqdm(paths):
    # print(path)
    with open(path) as f:
        d = json.load(f)
    i = path.rindex('\\')

    no2 = [x['components']['no2'] for x in d['list']]
    so2 = [x['components']['so2'] for x in d['list']]
    co = [x['components']['co'] for x in d['list']]
    o3 = [x['components']['o3'] for x in d['list']]
    pm10 = [x['components']['pm10'] for x in d['list']]
    pm25 = [x['components']['pm2_5'] for x in d['list']]
    dt = [x['dt'] for x in d['list']]

    df = pd.DataFrame({'datetime_unix': dt,
                    'o3': o3,
                    'pm25': pm25,
                    'pm10': pm10,
                    'co': co,
                    'so2': so2,
                    'no2': no2})
    df['lon'] = d['coord']['lon']
    df['lat'] = d['coord']['lat']
    df['location'] = path[i+1:-10]
    dfs.append(df)

full_df = pd.concat(dfs, axis=0).reset_index(drop=True)
full_df['timestamp'] = pd.to_datetime(full_df['datetime_unix'],unit='s')
full_df['timestamp'] = full_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Asia/Ho_Chi_Minh')
full_df = full_df.drop(['datetime_unix'], axis=1)
full_df = full_df[['location', 'lon', 'lat', 'timestamp',
                   'o3', 'pm25', 'pm10', 'co', 'so2', 'no2']]
full_df = full_df.sort_values(by=['location', 'timestamp']).reset_index(drop=True)
full_df

C:\Users\sonle\AppData\Local\Temp\ipykernel_38400\1850091446.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for path in tqdm(paths):


  0%|          | 0/161 [00:00<?, ?it/s]

,location,lon,lat,timestamp,o3,pm25,pm10,co,so2,no2
0,SRB.1.1_1,22.0542,44.0709,2023-01-01 00:00:00+07:00,28.61,10.45,13.44,210.29,0.09,3.81
1,SRB.1.1_1,22.0542,44.0709,2023-01-01 01:00:00+07:00,31.83,10.58,13.69,213.62,0.11,3.51
2,SRB.1.1_1,22.0542,44.0709,2023-01-01 02:00:00+07:00,35.05,10.60,13.87,216.96,0.11,3.17
3,SRB.1.1_1,22.0542,44.0709,2023-01-01 03:00:00+07:00,36.48,10.61,14.09,216.96,0.13,2.91
4,SRB.1.1_1,22.0542,44.0709,2023-01-01 04:00:00+07:00,37.91,10.48,14.07,216.96,0.16,2.70
...,...,...,...,...,...,...,...,...,...,...
1385860,SRB.9.4_1,20.2648,43.7392,2023-12-30 20:00:00+07:00,43.99,9.86,12.76,250.34,1.09,4.97
1385861,SRB.9.4_1,20.2648,43.7392,2023-12-30 21:00:00+07:00,38.27,9.90,13.14,247.00,0.57,6.00
1385862,SRB.9.4_1,20.2648,43.7392,2023-12-30 22:00:00+07:00,35.05,9.37,12.95,243.66,0.35,6.77
1385863,SRB.9.4_1,20.2648,43.7392,2023-12-30 23:00:00+07:00,32.54,9.23,13.18,243.66,0.27,7.71


In [25]:
full_df['year'] = full_df['timestamp'].dt.year
full_df['date'] = full_df['timestamp'].dt.date
full_df

,location,lon,lat,timestamp,o3,pm25,pm10,co,so2,no2,year,date
0,SRB.1.1_1,22.0542,44.0709,2023-01-01 00:00:00+07:00,28.61,10.45,13.44,210.29,0.09,3.81,2023,2023-01-01
1,SRB.1.1_1,22.0542,44.0709,2023-01-01 01:00:00+07:00,31.83,10.58,13.69,213.62,0.11,3.51,2023,2023-01-01
2,SRB.1.1_1,22.0542,44.0709,2023-01-01 02:00:00+07:00,35.05,10.60,13.87,216.96,0.11,3.17,2023,2023-01-01
3,SRB.1.1_1,22.0542,44.0709,2023-01-01 03:00:00+07:00,36.48,10.61,14.09,216.96,0.13,2.91,2023,2023-01-01
4,SRB.1.1_1,22.0542,44.0709,2023-01-01 04:00:00+07:00,37.91,10.48,14.07,216.96,0.16,2.70,2023,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
1385860,SRB.9.4_1,20.2648,43.7392,2023-12-30 20:00:00+07:00,43.99,9.86,12.76,250.34,1.09,4.97,2023,2023-12-30
1385861,SRB.9.4_1,20.2648,43.7392,2023-12-30 21:00:00+07:00,38.27,9.90,13.14,247.00,0.57,6.00,2023,2023-12-30
1385862,SRB.9.4_1,20.2648,43.7392,2023-12-30 22:00:00+07:00,35.05,9.37,12.95,243.66,0.35,6.77,2023,2023-12-30
1385863,SRB.9.4_1,20.2648,43.7392,2023-12-30 23:00:00+07:00,32.54,9.23,13.18,243.66,0.27,7.71,2023,2023-12-30


In [29]:
full_df = full_df.sort_values(by=['location', 'timestamp'])


def calculate_rolling_avg(group):
    # https://environment.ec.europa.eu/topics/air/air-quality/eu-air-quality-standards_en
    # https://uk-air.defra.gov.uk/air-pollution/faq?question=20#:~:text=The%20maximum%20daily%20running%208,average%20concentrations%20over%20a%20year. 

    group = group.sort_index()
    group['o3_8hr_avg'] = group['o3'].rolling(window=8, min_periods=1).mean()
    # group['co_8hr_avg'] = group['co'].rolling(window=8, min_periods=1).mean()
    return group

full_df = full_df.groupby('location').apply(calculate_rolling_avg).reset_index(drop=True)
o3_daily = full_df.groupby(['location', 'year', 'date']).agg(o3_daily=('o3_8hr_avg', 'max')).reset_index()
o3_yearly = o3_daily.groupby(['location', 'year']).agg(o3=('o3_daily', 'mean')).reset_index()
o3_yearly

C:\Users\sonle\AppData\Local\Temp\ipykernel_38400\1765404289.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  full_df = full_df.groupby('location').apply(calculate_rolling_avg).reset_index(drop=True)


,location,year,o3
0,SRB.1.1_1,2023,81.761742
1,SRB.1.2_1,2023,83.037593
2,SRB.1.3_1,2023,80.358395
3,SRB.1.4_1,2023,77.777445
4,SRB.10.1_1,2023,83.516670
...,...,...,...
156,SRB.8.8_1,2023,84.917634
157,SRB.9.1_1,2023,77.803461
158,SRB.9.2_1,2023,76.597913
159,SRB.9.3_1,2023,76.742917


In [30]:
pm25_yearly = full_df.groupby(['location', 'year']).agg(pm25=('pm25', 'mean')).reset_index()
pm25_yearly

,location,year,pm25
0,SRB.1.1_1,2023,7.701916
1,SRB.1.2_1,2023,7.652704
2,SRB.1.3_1,2023,8.204908
3,SRB.1.4_1,2023,8.413564
4,SRB.10.1_1,2023,9.394835
...,...,...,...
156,SRB.8.8_1,2023,10.187171
157,SRB.9.1_1,2023,8.329545
158,SRB.9.2_1,2023,9.933960
159,SRB.9.3_1,2023,6.042374


In [31]:
out_df = pd.merge(pm25_yearly, o3_yearly, on=['location', 'year'], how='inner')
out_df = out_df.rename({'location': 'GID_2'}, axis=1)
out_df

,GID_2,year,pm25,o3
0,SRB.1.1_1,2023,7.701916,81.761742
1,SRB.1.2_1,2023,7.652704,83.037593
2,SRB.1.3_1,2023,8.204908,80.358395
3,SRB.1.4_1,2023,8.413564,77.777445
4,SRB.10.1_1,2023,9.394835,83.516670
...,...,...,...,...
156,SRB.8.8_1,2023,10.187171,84.917634
157,SRB.9.1_1,2023,8.329545,77.803461
158,SRB.9.2_1,2023,9.933960,76.597913
159,SRB.9.3_1,2023,6.042374,76.742917


In [37]:
out_df.to_csv('D:\Work\WB\LDT\countries\SRB\datasets\SRB_air_pollution_2023_GID2.csv', index=False)

In [32]:
# gb = full_df.groupby(['location', 'year']).agg({'pm25': 'mean',
#                                            'pm10': 'mean',
#                                            'no2': 'mean'}).reset_index()
# gb = gb.rename({'location': 'NUTS_ID'}, axis=1)
# gb = gdf[['NUTS_ID', 'NAME_LATN']].merge(gb, on=['NUTS_ID'], how='left')
# gb = gb.rename({'NAME_LATN': 'DISTRICT'}, axis=1)
# gb.columns = gb.columns.str.upper()
# gb['UNIT'] = 'ug/m3'
# gb

In [33]:
# gb.to_csv('countries/SRB/datasets/SRB_air_pollution_2023_NUTS3.csv', index=False)

In [34]:
# gdp = pd.read_csv('countries/SRB/data/gdp/EU_GDP_2021_PPP.csv')
# gdp

In [124]:
# srb_gdp = gdp[(gdp['NUTS_ID'].str.contains('RS')) & (gdp['LEVL_CODE'] == 3)]
# srb_gdp = srb_gdp[srb_gdp['NUTS_ID'] != 'RSZZZ']
# srb_gdp = srb_gdp[srb_gdp['UNIT'] == 'Million EUR']
# srb_gdp = srb_gdp[srb_gdp['YEAR'] == 2023].reset_index(drop=True)
# srb_gdp['VALUE'] = srb_gdp['VALUE']*1000000
# srb_gdp['UNIT'] = 'EUR 2021'
# srb_gdp = pd.merge(srb_gdp, gdf[['NUTS_ID', 'NAME_LATN']], on=['NUTS_ID'], how='left')
# srb_gdp = srb_gdp.rename({'VALUE': 'GDP',
#                           'NAME_LATN': 'DISTRICT'}, axis=1)
# srb_gdp = srb_gdp[['NUTS_ID', 'DISTRICT', 'YEAR', 'GDP', 'UNIT']]
# srb_gdp.head()

,NUTS_ID,DISTRICT,YEAR,GDP,UNIT
0,RS110,City of Belgrade,2023,2.788723e+10,EUR 2021
1,RS121,Zapadnobačka oblast,2023,1.272390e+09,EUR 2021
2,RS122,Južnobanatska oblast,2023,2.453800e+09,EUR 2021
3,RS123,Južnobačka oblast,2023,7.281000e+09,EUR 2021
4,RS124,Severnobanatska oblast,2023,1.055790e+09,EUR 2021


In [35]:
# srb_gdp.to_csv('countries/SRB/datasets/SRB_gdp_2023_NUTS3.csv', index=False)